# A short notebook to show the various prompts

In [3]:
import numpy as np
import sys

sys.path.append("../../")    # Add the path to the root directory (where we can find the folder .git)

%load_ext autoreload
%autoreload 2 

from narval.utils import get_data_dir, FileSystem
from narval.pagefinder import PageFinder
from narval.prompts import get_prompt, NO_ANSWER_TAG

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Utility functions

In [4]:
FS = FileSystem()
DATA_DIR = get_data_dir()

# Adapted from format_prompt in pipeline.py and qamodel.py
def format_prompt(
        question, context, competence, year, collectivity, indicator, prompt_version, indicator_file
    ):
        """
        Returns the prompt corresponding to the input question, context,
        competence, year, collectivity and indicator for a given prompt_version
        """
        indicator_df = FS.read_csv_to_df(
            DATA_DIR + "/data/input/" + indicator_file,
            usecols=["code_ip", "unit_tag", "prompt_instruction"],
        )
        df = indicator_df.query("code_ip==@indicator")
        # Get the indictor unit (used in the prompt)
        unit_tag = df["unit_tag"].values[0]
        # Get specific prompt instructions for the indicator
        specific_instruction = (
            df["prompt_instruction"]
            .replace(np.nan, "")
            .values[0]
            .format(
                no_answer_tag=NO_ANSWER_TAG,
                unit_tag=unit_tag,
                year = str(year),
                year_plus_1=str(int(year) + 1),
            )
        )
        # Format the prompt
        prompt_params = {
            "context": context,
            "question": question,
            "unit_tag": unit_tag,
            "specific_instruction": specific_instruction,
            "year": year,
            "competence": competence,
            "collectivity": collectivity,
        }

        if "T5" in prompt_version:  # T5 prompts
            content = get_prompt(prompt_params, version=prompt_version)
            prompt = content, ""
        elif len(prompt_version)==2:  # Llama prompts
            system_content = get_prompt(prompt_params, version=prompt_version[0])
            user_content = get_prompt(prompt_params, version=prompt_version[1])
            prompt = system_content, user_content
        else:
             raise ValueError(f"The prompt version {prompt_version} is unknown")

        return prompt


### Choose prompt version 

In [5]:
question_file = "question_keyword_v7.csv"
indicator_file = "indicateurs_v6.csv"
prompt_version = ("Llama_prompt_system_v7", "Llama_prompt_user_v7")
#prompt_version = "T5_prompt_v1"

### Choose an indicator

In [6]:
competence = "assainissement collectif"

pagefinder = PageFinder(DATA_DIR + "/data/input/"+ question_file, competence)
question_df = pagefinder.keyword_df

In [7]:
question_df['indic'].unique()

array(['P205.3', 'P203.3', 'P204.3', 'P253.2', 'P202.2B', 'P255.3',
       'P258.1', 'D202.0', 'P206.3', 'D203.0', 'P201.1', 'P254.3',
       'P207.0', 'P256.2', 'D204.0', 'P257.0', 'P252.2', 'P251.1',
       'D201.0'], dtype=object)

### Choose a question

In [11]:
indicator = 'D203.0'

sub_df = question_df[question_df["indic"]==indicator]
sub_df

,question,mot,indic,competence
16,Quelle est la valeur de l'indicateur D203.0,D203.0,D203.0,assainissement collectif
17,Quelle est la quantité de boues évacuées (D203...,boue,D203.0,assainissement collectif


In [12]:
question = sub_df.loc[17, "question"]
question

'Quelle est la quantité de boues évacuées (D203.0), et non pas la quantité de boues produites,'

### Show prompt

In [13]:
context = "blablabla blablabla blablabla"
year = "2021"
collectivity = "MaVille"

prompt = format_prompt(
        question, context, competence, year, collectivity, indicator, prompt_version, indicator_file
    )

for i in range(len(prompt)):
    print(prompt[i])


Tu es un assistant administratif qui répond à des questions sur les services d'assainissement collectif en France.
Tu dois extraire la valeur d'un indicateur à partir d'extraits d'un rapport sur l'assainissement collectif en 2021 dans la collectivité "MaVille".
-------
Instructions : 
- La valeur de l'indicateur à trouver est un nombre exprimé en tonnes de matières sèches (tMS). 
- Si tu ne trouves pas la réponse pour l'année 2021 dans l'extrait, réponds "je ne trouve pas".
- Si tu n'as pas assez d'information dans l'extrait pour répondre, réponds "je ne trouve pas".
- Sois le plus concis possible. Ta réponse doit être uniquement un nombre (dans les bonnes unités) ou "je ne trouve pas".
-------
Extraits : blablabla blablabla blablabla


Question : Quelle est la quantité de boues évacuées (D203.0), et non pas la quantité de boues produites, en 2021 ?

